<a href="https://colab.research.google.com/github/arnaud22560/Titanic_Disaster/blob/main/Titanic_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic - Machine Learning from Disaster
Predict survival on the Titanic and get familiar with ML basics

[Kaggle competition](https://www.kaggle.com/c/titanic/overview)

![alt text](https://github.com/adam-p/markdown-here/raw/master/src/common/images/icon48.png "Logo Title Text 1")
